In [1]:
PROJECT_DIR=`git rev-parse --show-toplevel`
cd $PROJECT_DIR/microbial
ls

all_kat_hist                            freq_k7.hist
all_kat_hist.dist_analysis.json         images
all_kat_hist.png                        kat.hist
bigsi                                   kat.hist.dist_analysis.json
data                                    kat.hist.png
ERR1144974                              microbial-bigsi.ipynb
ERR1144974.dist_analysis.json           microbial-genomes.txt
ERR1144974_kat_hist                     microbial-process-data.ipynb
ERR1144974_kat_hist.dist_analysis.json  microbial-process-data.ipynb.bak
ERR1144974_kat_hist.png                 o.png
ERR1144974.png                          sha256
freq_k31.hist


# Step 1: Download data

As a first step, let's download all the microbial genomes in the file `microbial-genomes.txt`.

In [1]:
mkdir data

for name in `cat microbial-genomes.txt`;
do
    set -x
    fasterq-dump -o data/${name} --split-files ${name}
    pigz data/${name}*
done
set +x

mkdir: cannot create directory ‘data’: File exists
+ fasterq-dump -o data/SRR10519619 --split-files SRR10519619
spots read      : 2,465,304
reads read      : 4,930,608
reads written   : 4,930,608
+ pigz data/SRR10519619_1.fastq data/SRR10519619_2.fastq
+ for name in `cat microbial-genomes.txt`
+ set -x
+ fasterq-dump -o data/SRR10519637 --split-files SRR10519637
spots read      : 7,316,985
reads read      : 14,633,970
reads written   : 14,633,970
+ pigz data/SRR10519637_1.fastq data/SRR10519637_2.fastq
+ for name in `cat microbial-genomes.txt`
+ set -x
+ fasterq-dump -o data/SRR10519620 --split-files SRR10519620
spots read      : 3,443,404
reads read      : 6,886,808
reads written   : 6,886,808
+ pigz data/SRR10519620_1.fastq data/SRR10519620_2.fastq
+ for name in `cat microbial-genomes.txt`
+ set -x
+ fasterq-dump -o data/SRR10519617 --split-files SRR10519617
spots read      : 2,681,834
reads read      : 5,363,668
reads written   : 5,363,668
+ pigz data/SRR10519617_1.fastq data/SRR105

In [6]:
ls data

ERR1144974_1.fastq.gz  SRR10298903_1.fastq.gz  SRR10519619_1.fastq.gz
ERR1144974_2.fastq.gz  SRR10298903_2.fastq.gz  SRR10519619_2.fastq.gz
ERR1144975_1.fastq.gz  SRR10298904_1.fastq.gz  SRR10519620_1.fastq.gz
ERR1144975_2.fastq.gz  SRR10298904_2.fastq.gz  SRR10519620_2.fastq.gz
ERR1144976_1.fastq.gz  SRR10298905_1.fastq.gz  SRR10519637_1.fastq.gz
ERR1144976_2.fastq.gz  SRR10298905_2.fastq.gz  SRR10519637_2.fastq.gz
ERR1144977_1.fastq.gz  SRR10298906_1.fastq.gz  SRR10521982_1.fastq.gz
ERR1144977_2.fastq.gz  SRR10298906_2.fastq.gz  SRR10521982_2.fastq.gz
ERR1144978_1.fastq.gz  SRR10298907_1.fastq.gz  SRR10521983_1.fastq.gz
ERR1144978_2.fastq.gz  SRR10298907_2.fastq.gz  SRR10521983_2.fastq.gz
ERR3655992_1.fastq.gz  SRR10512964_1.fastq.gz  SRR10521984_1.fastq.gz
ERR3655992_2.fastq.gz  SRR10512964_2.fastq.gz  SRR10521984_2.fastq.gz
ERR3655994_1.fastq.gz  SRR10512965_1.fastq.gz  SRR10527348_1.fastq.gz
ERR3655994_2.fastq.gz  SRR10512965_2.fastq.gz  SRR10527348_2.fastq.gz
ERR3655996_1.fastq.g

In [7]:
du -sh data

20G	data


Awesome. We've downloaded and compressed all the microbial genomes.

# Step 2: Cleaning reads

Now that we have data downloaded, let's subsample our data (and mask low-quality bases) using the software [seqtk](https://github.com/lh3/seqtk).

Let's first set some basic environment variables.

In [2]:
threads=48

Now, let's clean and subsample the reads down to a max of 1 million per each read pair (since some files have a lot of reads).

In [3]:
mkdir data/subsample

max_reads=1000000

for file in data/*_1.fastq.gz
do
    name=`basename $file _1.fastq.gz`
    
    input1=data/${name}_1.fastq.gz
    input2=data/${name}_2.fastq.gz
    output1=data/subsample/${name}_1.fastq.gz
    output2=data/subsample/${name}_2.fastq.gz
    
    echo "Working on ${name}"
        
    conda run --name seqtk seqtk sample -s 41 $input1 ${max_reads} | pigz --fast -c -p ${threads} > ${output1}
    conda run --name seqtk seqtk sample -s 41 $input2 ${max_reads} | pigz --fast -c -p ${threads} > ${output2}
done

mkdir: cannot create directory ‘data/subsample’: File exists
Working on ERR1144974
Working on ERR1144975
Working on ERR1144976
Working on ERR1144977
Working on ERR1144978
Working on ERR3655992
Working on ERR3655994
Working on ERR3655996
Working on ERR3655998
Working on ERR3656002
Working on ERR3656004
Working on ERR3656010
Working on ERR3656012
Working on ERR3656013
Working on ERR3656015
Working on ERR3656018
Working on ERR3656019
Working on SRR10298903
Working on SRR10298904
Working on SRR10298905
Working on SRR10298906
Working on SRR10298907
Working on SRR10512964
Working on SRR10512965
Working on SRR10512968
Working on SRR10513325
Working on SRR10513326
Working on SRR10513328
Working on SRR10513363
Working on SRR10513672
Working on SRR10519468
Working on SRR10519469
Working on SRR10519616
Working on SRR10519617
Working on SRR10519619
Working on SRR10519620
Working on SRR10519637
Working on SRR10521982
Working on SRR10521983
Working on SRR10521984
Working on SRR10527348
Working on SR

In [6]:
du -sh data/subsample/

9.6G	data/subsample/


Awesome. Looks like we've cut our total data size by half.

# Step 3: Examining data

Now, let's use the software [dashing](https://github.com/dnbaker/dashing) (version `0.4.0`) to estimate the total number of unique k-mers in our dataset.

This sofware must be downloaded and placed in your `PATH`.

In [3]:
dashing_s512 --version
dashing_s512 hll -k 31 -p 32 data/*.fastq.gz

Dashing version: v0.4.0-5-g9c13
Dashing version: v0.4.0-5-g9c13
Dashing version: v0.4.0-5-g9c13
[int bns::hll_main(int, char**)] Processing 100 paths with 32 threads
Estimated number of unique exact matches: 1691459836.000000


This result tells us that among all our genomes, we have appromximetly 1.7*10^9 (1.7 billion) unique 31-mers in our dataset.

Let's normalize this value based on the number of bases in our data to get a value of **unique k-mers/hundred bp**.

First, we have to count all the basepairs in our dataset.

In [2]:
# Code derived from https://bioinformatics.stackexchange.com/a/966
pigz -dc data/*.fastq.gz | awk 'NR%4==2{c++; l+=length($0)}
          END{
                print "Number of reads: "c;
                print "Number of bases in reads: "l
              }'

Number of reads: 307223608
Number of bases in reads: 35864044717


Next we have to convert to our normalized value **unique k-mers/hundred bp**.

In [13]:
KMERS=1691459836
BP=35864044717
echo "${KMERS}/(${BP}/100)" | bc -l

4.71631086049317564484


Awesome. This dataset has about 4.7 unique 31-mers/100 bp.

Let's now look at a k-mer histogram using [kat](https://kat.readthedocs.io). This can be installed from bioconda.

In [11]:
conda run --name kat kat hist -o ERR1144974_kat_hist -t 32 -m 31 data/ERR1144974*.gz

Kmer Analysis Toolkit (KAT) V2.4.2

Running KAT in HIST mode
------------------------

Input 1 is a sequence file.  Counting kmers for input 1 (data/ERR1144974_1.fastq.gz data/ERR1144974_2.fastq.gz) ... done.  Time taken: 37.9s

Bining kmers ... done.  Time taken: 0.3s

Merging counts ... done.  Time taken: 0.0s

Saving results to disk ... done.  Time taken: 0.0s

Creating plot ... done.  Time taken: 0.8s

Analysing peaks
---------------

Analysing distributions for: ERR1144974_kat_hist ... done.  Time taken:  0.2s

K-mer frequency spectra statistics
----------------------------------
K-value used: 31
Peaks in analysis: 1
Global minima @ Frequency=36x (797)
Global maxima @ Frequency=499x (16869)
Overall mean k-mer frequency: 250x

  Index    Left    Mean    Right    StdDev    Max    Volume  Description
-------  ------  ------  -------  --------  -----  --------  -------------
      1       3     251      499       124    450    134040  1/2X

Calculating genome statistics
--------------

Awesome. We'll have to resize this image, so let's use `convert` from imagemagick to resize so we can display inline.

In [12]:
ls
convert ERR1144974_kat_hist.png -resize 600x images/ERR1144974_kat_hist.png

data                                    images
ERR1144974                              kat.hist
ERR1144974.dist_analysis.json           kat.hist.dist_analysis.json
ERR1144974_kat_hist                     kat.hist.png
ERR1144974_kat_hist.dist_analysis.json  microbial-genomes.txt
ERR1144974_kat_hist.png                 o.png
ERR1144974.png                          process-data.ipynb


The final k-mer distribution is:

![ERR1144974_kat_hist.png](images/ERR1144974_kat_hist.png)

Okay. Now let's try to look at a k-mer historygram over all files.

We first have to concatenate all files together.

In [1]:
mkdir data/all
cat data/*.fastq.gz > data/all/all.fastq.gz

Now, let's create a histogram over all this data.

In [2]:
conda run --name kat kat hist -o all_kat_hist -t 32 -m 31 data/all/all.fastq.gz

Kmer Analysis Toolkit (KAT) V2.4.2

Running KAT in HIST mode
------------------------

Input 1 is a sequence file.  Counting kmers for input 1 (data/all/all.fastq.gz) ...
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!

 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!

 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 success!
 succes

Hooray, it finished :). Now let's resize the image and remove the 20GB all fastq file.

In [6]:
ls
convert all_kat_hist.png -resize 600x images/all_kat_hist.png
rm data/all/all.fastq.gz

all_kat_hist                            ERR1144974.png
all_kat_hist.dist_analysis.json         images
all_kat_hist.png                        kat.hist
data                                    kat.hist.dist_analysis.json
ERR1144974                              kat.hist.png
ERR1144974.dist_analysis.json           microbial-genomes.txt
ERR1144974_kat_hist                     o.png
ERR1144974_kat_hist.dist_analysis.json  process-data.ipynb
ERR1144974_kat_hist.png


![all_kat_hist.png](images/all_kat_hist.png)

Huh, interesting.

In [13]:
conda run --name squeakr squeakr count -e -k 31 -s 20 -t 24 -o tmp.squeakr data/ERR1144974_1.fastq.gz



Parsing command line failed with exception: The required input directory tmp.squeakr does not seem to exist.


SYNOPSIS
        squeakr count [-e] -k <k-size> [-c <cutoff>] [-n] [-s <log-slots>] [-t <num-threads>] -o <out-file> <files>... [-v]
        squeakr query -f <squeakr-file> -q <query-file> -o <output-file> [-v]
        squeakr inner_prod <first-input> <second-input> [-v]
        squeakr list -f <squeakr-file> -o <output-file> [-v]
        squeakr info -f <squeakr-file> [-v]
        squeakr help [-v]

OPTIONS
        -e, --exact squeakr-exact (default is Squeakr approximate)
        <k-size>    length of k-mers to count
        <cutoff>    only output k-mers with count greater than or equal to cutoff (default = 1)

        -n, --no-counts
                    only output k-mers and no counts (default = false)

        <log-slots> log of number of slots in the CQF. (Size argument is only optional when numthreads is exactly 1.)

        <num-threads>
                    number o

: 1